In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pickle
from tqdm import tqdm_notebook as tqdm
from scipy.special import erf
from domain_model import DocumentManagement
from simulation import ACC, idm_approaching_pars, IDMPlus, acc_approaching_pars, \
    SimulationApproaching

## Show number of scenarios found

In [ ]:
leaddec = DocumentManagement(os.path.join("data", "5_scenarios", "lead_braking2.json"))
print("Number of lead vehicle decelerating scenarios: {:d}"
      .format(len(leaddec.collections["scenario"])))

In [ ]:
approaching = DocumentManagement(os.path.join("data", "5_scenarios", "approaching_vehicle2.json"))
print("Number of approaching slower vehicle scenarios: {:d}"
      .format(len(approaching.collections["scenario"])))

In [ ]:
cutins = DocumentManagement(os.path.join("data", "5_scenarios", "cut_in_scenarios2.json"))
print("Number of cut-in scenarios: {:d}".format(len(cutins.collections["scenario"])))

## Simulations

#### Approaching stopped vehicle (false positive) with 2 vehicles

In [ ]:
s = SimulationApproaching([ACC(), IDMPlus()], [acc_approaching_pars, idm_approaching_pars],
                          min_simulation_time=5)

In [ ]:
s.simulation(dict(vego=20, ratio_vtar_vego=0, amin=-6, init_position=20, reactiontime=13),
             plot=True, ignore_stop=[True, False])

## Estimating risk for false positives

A vehicle is detected at distance X
Ego is at speed V
Vehicle behind with same speed (modelled using IDM+, with delay)

In [ ]:
mu_lognormal = np.log(.92**2/np.sqrt(.92**2+.28**2))
sigma_lognormal = np.sqrt(np.log(1+.28**2/.92**2))
def cdf(x):
    return 0.5*(1 + erf((np.log(x) - mu_lognormal) / (sigma_lognormal*np.sqrt(2))))

In [ ]:
simulation = SimulationApproaching([ACC(), IDMPlus()], 
                                   [acc_approaching_pars, idm_approaching_pars],
                                   min_simulation_time=5)
def fp_simulation(speed, distance, reactiontime, thw):
     return simulation.simulation(dict(vego=speed, ratio_vtar_vego=0, thw=thw,
                                       amin=-6, init_position=distance, reactiontime=reactiontime),
                                  ignore_stop=[True, False])

def is_collision(speed, distance, reactiontime, thw):
    result = fp_simulation(speed, distance, reactiontime, thw)
    if result[1] < 0.0:
        return True
    return False

In [ ]:
def prob_of_collision(speed, distance, thw):
    minimum = 0.3
    maximum = 4
    if is_collision(speed, distance, minimum, thw):
        return 1.0
    if not is_collision(speed, distance, maximum, thw):
        return 0.0
    while cdf(maximum) - cdf(minimum) > 0.00001:
        if is_collision(speed, distance, (maximum + minimum) / 2, thw):
            maximum = (maximum + minimum) / 2
        else:
            minimum = (maximum + minimum) / 2
    return 1 - cdf((maximum + minimum) / 2)

In [ ]:
def make_heatplot_fp(dmin, dmax, vmin, vmax, thw=1.0, overwrite=False):
    name = "fp_prob_{:d}_{:d}_{:d}_{:d}_{:02.0f}".format(dmin, dmax, vmin, vmax, 10*thw)
    filename = os.path.join("data", "7_simulation_results", "{:s}.p".format(name))
    
    distances = np.linspace(dmin, dmax, 50)
    speeds = np.linspace(vmin, vmax, 50)
        
    if not overwrite and os.path.exists(filename):
        with open(filename, "rb") as file:
            prob_collision = pickle.load(file)
    else:
        prob_collision = np.zeros((len(distances), len(speeds)))
        for i, distance in enumerate(tqdm(distances)):
            for j, speed in enumerate(speeds):
                prob_collision[i, j] = prob_of_collision(speed, distance, thw)
        with open(filename, "wb") as file:
            pickle.dump(prob_collision, file)
    
    f, ax = plt.subplots(1, 1)
    heatmap = plt.contourf(speeds, distances, prob_collision, cmap='hot')
    cbar = f.colorbar(heatmap)
    plt.title("THW of follower is {:3.1f} s".format(thw))
    plt.xlabel("Speed [m/s]")
    plt.ylabel("Initial distance of false positive [m]")
    cbar.ax.set_ylabel("Probability of collision rear vehicle")
    
    f.savefig(os.path.join("figs", "{:s}.png".format(name)))

False P (resulting from triggering condition)
 - Obtain impact speed heatmap
False N (resulting from triggering condition)  (3 -> 2 (ego) -> 1 (stationary, not detection))
 - Focus on this one

Low-mu (triggering condition) -> lead vehicle braking, approaching, cut-in, cut-out (scenario)
Slope (triggering condition) -> lead vehicle braking, approaching, cut-in, cut-out (scenario)

gap(t) = inital_gap(thw=1.1s) - 0.5*a*t^2
Make heatmap of impact speed

In [ ]:
make_heatplot_fp(10, 100, 5, 40, 1.0)

In [ ]:
make_heatplot_fp(10, 100, 5, 40, 0.6)

In [ ]:
make_heatplot_fp(10, 100, 5, 40, 0.3)

#### Heatmap for impact speed

In [ ]:
def find_reaction_time_collision(speed, distance, thw):
    minimum = 0.1
    maximum = 10
    if not is_collision(speed, distance, maximum, thw):
        return -1
    if is_collision(speed, distance, minimum, thw):
        return minimum
    while (maximum - minimum) >= 0.01:
        if is_collision(speed, distance, (maximum+minimum)/2, thw):
            maximum = (maximum + minimum) / 2
        else:
            minimum = (maximum + minimum) / 2
    return maximum

In [ ]:
def get_new_cdf(xmin, xmax=5):
    x = np.linspace(xmin, xmax, 100)
    y = np.array([cdf(xx) for xx in x])
    y -= y[0]
    y /= y[-1]
    return x, y

In [ ]:
def make_heatplot_fp_vimpact(dmin, dmax, vmin, vmax, thw=1.0, overwrite=False):
    name = "fp_vimpact_{:d}_{:d}_{:d}_{:d}_{:02.0f}".format(dmin, dmax, vmin, vmax, 10*thw)
    filename = os.path.join("data", "7_simulation_results", "{:s}.p".format(name))
    
    distances = np.linspace(dmin, dmax, 50)
    speeds = np.linspace(vmin, vmax, 50)
    n_monte_carlo = 20
        
    if not overwrite and os.path.exists(filename):
        with open(filename, "rb") as file:
            vimpact = pickle.load(file)
    else:
        np.random.seed(0)
        vimpact = np.zeros((len(distances), len(speeds), n_monte_carlo))
        for i, distance in enumerate(tqdm(distances)):
            for j, speed in enumerate(speeds):
                min_reaction_time = find_reaction_time_collision(speed, distance, thw)
                if min_reaction_time < 0:
                    continue
                cdfx, cdfy = get_new_cdf(min_reaction_time)
                for k in range(n_monte_carlo):
                    reaction_time = np.interp(np.random.rand(), cdfy, cdfx)
                    result = fp_simulation(speed, distance, reaction_time, thw)[1]
                    if result < 0:  # Only use result if there is an impact.
                        vimpact[i, j, k] = -result
        with open(filename, "wb") as file:
            pickle.dump(vimpact, file)
    
    f, ax = plt.subplots(1, 1)
    heatmap = plt.contourf(speeds, distances, np.mean(vimpact, axis=2), cmap='hot')
    cbar = f.colorbar(heatmap)
    plt.title("THW of follower is {:3.1f} s".format(thw))
    plt.xlabel("Speed [m/s]")
    plt.ylabel("Initial distance of false positive [m]")
    cbar.ax.set_ylabel("Expected impact speed with collision [m/s]")
    
    f.savefig(os.path.join("figs", "{:s}.png".format(name)))

In [ ]:
make_heatplot_fp_vimpact(10, 100, 5, 40, 1.0)

In [ ]:
make_heatplot_fp_vimpact(10, 100, 5, 40, 0.6)

In [ ]:
make_heatplot_fp_vimpact(10, 100, 5, 40, 0.3)

## Estimating risk for false negatives